# NonBDNAFinder - Single Cell Analysis

**Quick Start:** Run this single cell to analyze DNA sequences from file(s) and generate results.

### Instructions:
1. **Modify input:** Set `file_patterns` to your FASTA file(s) - can be a single file, list of files, or glob pattern
2. **Run cell:** Execute this cell (Shift+Enter)
3. **Check output:** Results saved to CSV files with prefix specified in `output_prefix`

### Input Options:
- Single file: `file_patterns = 'genome.fasta'`
- Multiple files: `file_patterns = ['file1.fasta', 'file2.fasta']`
- Glob pattern: `file_patterns = 'data/*.fasta'` or `'data/**/*.fa'`

### Output:
- `{output_prefix}_results.csv` - All detected motifs
- `{output_prefix}_summary.csv` - Summary statistics by motif class

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION - Edit these settings
# ═══════════════════════════════════════════════════════════════════════════════

# Input: Single file, list of files, or glob pattern (e.g., '*.fasta', 'data/**/*.fa')
file_patterns = 'example.fasta'  # EDIT THIS

# Output prefix for result files
output_prefix = 'nonbdna'  # EDIT THIS (optional)

# ═══════════════════════════════════════════════════════════════════════════════
# ANALYSIS CODE - Run as-is
# ═══════════════════════════════════════════════════════════════════════════════

import sys, os, glob, pandas as pd
from pathlib import Path

# Add repository to path for local execution
repo_root = Path.cwd()
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

# Import NonBDNAFinder
from Utilities.nonbscanner import analyze_sequence
from Utilities.utilities import parse_fasta

# Resolve file patterns to list of files
def resolve_files(patterns):
    """Convert file pattern(s) to list of file paths"""
    if isinstance(patterns, str):
        patterns = [patterns]
    files = []
    for pattern in patterns:
        if '*' in pattern or '?' in pattern:
            files.extend(glob.glob(pattern, recursive=True))
        else:
            files.append(pattern)
    return [f for f in files if os.path.isfile(f)]

# Get files to process
input_files = resolve_files(file_patterns)
if not input_files:
    print(f"❌ No files found matching pattern: {file_patterns}")
    print("\nPlease check:")
    print("  1. File path is correct")
    print("  2. File exists in the specified location")
    print("  3. Glob pattern syntax is correct (e.g., '*.fasta', 'data/**/*.fa')")
else:
    print(f"📁 Found {len(input_files)} file(s) to process")
    for f in input_files:
        print(f"   • {f}")
    print()
    
    # Process all files
    all_motifs = []
    for i, fasta_file in enumerate(input_files, 1):
        print(f"\n{'='*80}")
        print(f"Processing file {i}/{len(input_files)}: {fasta_file}")
        print('='*80)
        
        try:
            # Read FASTA file
            with open(fasta_file, 'r') as f:
                fasta_content = f.read()
            
            # Parse sequences
            sequences = parse_fasta(fasta_content)
            print(f"📊 Found {len(sequences)} sequence(s) in file")
            
            # Analyze each sequence
            for seq_name, seq in sequences.items():
                print(f"\n  🔬 Analyzing: {seq_name} ({len(seq):,} bp)")
                
                # Run analysis
                motifs = analyze_sequence(seq, seq_name)
                
                # Add file source
                for motif in motifs:
                    motif['Source_File'] = os.path.basename(fasta_file)
                
                all_motifs.extend(motifs)
                print(f"  ✓ Detected {len(motifs)} motifs")
        
        except Exception as e:
            print(f"  ❌ Error processing {fasta_file}: {str(e)}")
            continue
    
    # Generate results
    if all_motifs:
        print(f"\n\n{'='*80}")
        print("📊 ANALYSIS COMPLETE")
        print('='*80)
        
        # Convert to DataFrame
        df = pd.DataFrame(all_motifs)
        
        # Summary statistics
        print(f"\n✓ Total motifs detected: {len(df):,}")
        print(f"✓ Motif classes found: {df['Class'].nunique()}")
        print("\nMotifs by class:")
        class_counts = df['Class'].value_counts()
        for motif_class, count in class_counts.items():
            print(f"  • {motif_class}: {count:,}")
        
        # Save detailed results
        results_file = f"{output_prefix}_results.csv"
        df.to_csv(results_file, index=False)
        print(f"\n💾 Detailed results saved to: {results_file}")
        
        # Save summary by class
        summary = df.groupby('Class').agg({
            'Start': 'count',
            'Length': ['mean', 'min', 'max'],
            'Score': ['mean', 'min', 'max']
        }).round(2)
        summary.columns = ['Count', 'Avg_Length', 'Min_Length', 'Max_Length', 'Avg_Score', 'Min_Score', 'Max_Score']
        summary_file = f"{output_prefix}_summary.csv"
        summary.to_csv(summary_file)
        print(f"💾 Summary statistics saved to: {summary_file}")
        
        print("\n✨ Analysis complete! Check the output files for results.")
    else:
        print("\n⚠️  No motifs detected in any of the analyzed sequences.")
        print("This could mean:")
        print("  • Sequences are too short")
        print("  • Sequences lack Non-B DNA forming patterns")
        print("  • Input files were empty or invalid")